# Sequence classification model for IMDB Sentiment Analysis
(c) Deniz Yuret, 2018

* Objectives: Learn the structure of the IMDB dataset and train a simple RNN model.
* Prerequisites: RNN models (06.rnn.ipynb), param, GRU, nll, minibatch, accuracy, Adam, train!
* Knet: dir (used by imdb.jl)

In [2]:
using Pkg
for p in ("Knet","ProgressMeter")
    haskey(Pkg.installed(),p) || Pkg.add(p)
end

true

In [48]:
EPOCHS=3          # Number of training epochs
BATCHSIZE=64      # Number of instances in a minibatch
EMBEDSIZE=125     # Word embedding size
NUMHIDDEN=100     # Hidden layer size
MAXLEN=150        # maximum size of the word sequence, pad shorter sequences, truncate longer ones
VOCABSIZE=30000   # maximum vocabulary size, keep the most frequent 30K, map the rest to UNK token
NUMCLASS=2        # number of output classes
DROPOUT=0.0       # Dropout rate
LR=0.001          # Learning rate
BETA_1=0.9        # Adam optimization parameter
BETA_2=0.999      # Adam optimization parameter
EPS=1e-08         # Adam optimization parameter

1.0e-8

## Load and view data

In [4]:
using Knet: Knet
ENV["COLUMNS"]=92                     # column width for array printing
include(Knet.dir("data","imdb.jl"))   # defines imdb loader

imdb

In [5]:
@doc imdb

```
imdb()
```

Load the IMDB Movie reviews sentiment classification dataset from https://keras.io/datasets and return (xtrn,ytrn,xtst,ytst,dict) tuple.

# Keyword Arguments:

  * url=https://s3.amazonaws.com/text-datasets: where to download the data (imdb.npz) from.
  * dir=Pkg.dir("Knet/data"): where to cache the data.
  * maxval=nothing: max number of token values to include. Words are ranked by how often they occur (in the training set) and only the most frequent words are kept. nothing means keep all, equivalent to maxval = vocabSize + pad + stoken.
  * maxlen=nothing: truncate sequences after this length. nothing means do not truncate.
  * seed=0: random seed for sample shuffling. Use system seed if 0.
  * pad=true: whether to pad short sequences (padding is done at the beginning of sequences). pad_token = maxval.
  * stoken=true: whether to add a start token to the beginning of each sequence. start_token = maxval - pad.
  * oov=true: whether to replace words >= oov*token with oov*token (the alternative is to skip them). oov_token = maxval - pad - stoken.


In [18]:
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=VOCABSIZE);

  5.380244 seconds (12.46 M allocations: 652.018 MiB, 5.34% gc time)


In [19]:
summary.((xtrn,ytrn,xtst,ytst,imdbdict))

("25000-element Array{Array{Int32,1},1}", "25000-element Array{Int8,1}", "25000-element Array{Array{Int32,1},1}", "25000-element Array{Int8,1}", "Dict{String,Int32} with 88584 entries")

In [20]:
# Words are encoded with integers
rand(xtrn)'

1×150 LinearAlgebra.Adjoint{Int32,Array{Int32,1}}:
 153  718  3  315  486  5  665  552  20  1  …  788  21  14  29998  18  15  1611  4371

In [21]:
# Each word sequence is padded or truncated to length 150
length.(xtrn)'

1×25000 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 150  150  150  150  150  150  150  150  150  …  150  150  150  150  150  150  150  150

In [52]:
# Define a function that can print the actual words:
imdbvocab = Array{String}(undef,length(imdbdict))
for (k,v) in imdbdict; imdbvocab[v]=k; end
imdbvocab[VOCABSIZE-2:VOCABSIZE] = ["<unk>","<s>","<pad>"]
function reviewstring(x,y=0)
    x = x[x.!=VOCABSIZE] # remove pads
    """$(("Sample","Negative","Positive")[y+1]) review:\n$(join(imdbvocab[x]," "))"""
end

reviewstring (generic function with 2 methods)

In [70]:
# Hit Ctrl-Enter to see random reviews:
r = rand(1:length(xtrn))
println(reviewstring(xtrn[r],ytrn[r]))

Negative review:
<s> i cant help it but i seem to like films that are meant to be scary and are just plain bad i have personally listed it in my own top 10 worst movies right under creatures of the abyss watch this film and have a laugh just don't expect to see any academy awards for acting more chance of understanding the film its self in all honesty though i have seen much worse than this plus some maniac cruising round the desert wiping the same people out that just died is that unbelievable that its got to be original i think its one of those love or hate movies you can make up your own mind yes its awful but it pulls it off somehow thats why i love it


In [32]:
# Here are the labels: 1=negative, 2=positive
ytrn'

1×25000 LinearAlgebra.Adjoint{Int8,Array{Int8,1}}:
 2  2  2  1  1  2  1  1  2  2  1  2  2  2  2  …  1  1  2  2  1  2  1  1  1  2  1  1  2  1

## Define the model

In [33]:
using Knet: param, dropout, RNN

In [34]:
struct SequenceClassifier; input; rnn; output; end

In [35]:
SequenceClassifier(input::Int, embed::Int, hidden::Int, output::Int) =
    SequenceClassifier(param(embed,input), RNN(embed,hidden,rnnType=:gru), param(output,hidden))

SequenceClassifier

In [36]:
function (sc::SequenceClassifier)(input; pdrop=0)
    embed = sc.input[:, permutedims(hcat(input...))]
    embed = dropout(embed,pdrop)
    hidden = sc.rnn(embed)
    hidden = dropout(hidden,pdrop)
    return sc.output * hidden[:,:,end]
end

## Experiment

In [38]:
using Knet: minibatch
dtrn = minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)
dtst = minibatch(xtst,ytst,BATCHSIZE)
length.((dtrn,dtst))

(390, 390)

In [44]:
# For running experiments
using Knet: train!, Adam, AutoGrad
import ProgressMeter

function trainresults(file,model)
    if (print("Train from scratch? ");readline()[1]=='y')
        updates = 0; prog = ProgressMeter.Progress(EPOCHS * length(dtrn))
        function callback(J)
            ProgressMeter.update!(prog, updates)
            return (updates += 1) <= prog.n
        end
        opt = Adam(lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS)
        train!(model, dtrn; callback=callback, optimizer=opt, pdrop=DROPOUT)
        Knet.gc()
        Knet.save(file,"model",model)
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model = Knet.load(file,"model")
    end
    return model
end

trainresults (generic function with 1 method)

In [49]:
using Knet: nll, accuracy
model = SequenceClassifier(VOCABSIZE,EMBEDSIZE,NUMHIDDEN,NUMCLASS)
nll(model,dtrn), nll(model,dtst), accuracy(model,dtrn), accuracy(model,dtst)

(0.6932078f0, 0.69321173f0, 0.4817708333333333, 0.48233173076923075)

In [50]:
model = trainresults("imdbmodel.jld2",model);

Train from scratch? stdin> y


Progress: 100%|█████████████████████████████████████████████████████| Time: 0:00:22


In [51]:
# 33s (0.059155148f0, 0.3877507f0, 0.9846153846153847, 0.8583733974358975)
nll(model,dtrn), nll(model,dtst), accuracy(model,dtrn), accuracy(model,dtst)

(0.04928492f0, 0.39949256f0, 0.9883413461538462, 0.8558894230769231)

## Playground

In [117]:
predictstring(x)="\nPrediction: " * ("Negative","Positive")[argmax(Array(vec(model([x]))))]
UNK = VOCABSIZE-2
str2ids(s::String)=[(i=get(imdbdict,w,UNK); i>=UNK ? UNK : i) for w in split(lowercase(s))]

str2ids (generic function with 1 method)

In [114]:
# Here we can see predictions for random reviews from the test set; hit Ctrl-Enter to sample:
r = rand(1:length(xtst))
println(reviewstring(xtst[r],ytst[r]))
println(predictstring(xtst[r]))

Negative review:
here that the people who write in detail about with a modicum of intelligence thoughtfulness and maturity tend to like at least a few things about this movie and rightly so in it's original cut most reasonable people i think would probably rate it at least 4 or 5 stars out of 10 five is average to me and i think this movie is about average for a sci fi pic br br in contrast to the above i've also noticed that the reviewers who seem immature dull and flip and as a result come off as <unk> from where i stand are the same ones who can't find anything good about this movie and basically trash it without cause based mostly on seeing it chopped up and on mst3k or if they have seen both cuts it seems they were greatly prejudiced by the mst3k viewing to begin with

Prediction: Negative


In [118]:
# Here the user can enter their own reviews and classify them:
println(predictstring(str2ids(readline(stdin))))

stdin> I can't recommend this movie .

Prediction: Negative
